# co-visitation matrix

In [1]:
import os
import sys
import gc
import subprocess
from dotenv import load_dotenv
load_dotenv
sys.path.append(os.getenv('UTILS_PATH'))
from tqdm import tqdm
import multiprocessing
import random
from collections import defaultdict

import pandas as pd
import cudf
import line_notify

In [2]:
SEED = 42
random.seed(SEED)

In [3]:
INPUT_DIR = os.getenv('INPUT_DIR')
OUTPUT_DIR = os.getenv('OUTPUT_DIR')
PREP_DIR = os.getenv("PREP_DIR")

In [4]:
CHUNK_N = 200

In [5]:
# データ読み込み
train_sessions = pd.read_pickle(PREP_DIR + "train_sessions.pkl")
test_sessions = pd.read_pickle(PREP_DIR + "test_sessions.pkl")

train_sessions["ts"] = (train_sessions["ts"]/1000).astype("int32")
test_sessions["ts"] = (test_sessions["ts"]/1000).astype("int32")

train_sessions = train_sessions[train_sessions["type"]=="orders"]
test_sessions = test_sessions[test_sessions["type"]=="orders"]

train_sessions = train_sessions.drop(columns=["type"])
test_sessions = test_sessions.drop(columns=["type"])

sessions = pd.concat([train_sessions, test_sessions])

# chunkのグループ分け
sessions["chunk"] = sessions["session"] % CHUNK_N

del train_sessions, test_sessions
gc.collect()

0

In [6]:
dfs = []
for _, chunk_df in tqdm(sessions.groupby("chunk"), total=CHUNK_N):
    df = cudf.from_pandas(chunk_df)
    df = df.merge(df, on="session")
    df = df.loc[((df.ts_x - df.ts_y).abs()< 24 * 60 * 60) & (df.aid_x != df.aid_y)]
    df = df.groupby(["aid_x", "aid_y"])["session"].count().reset_index()
    df = df.rename(columns = {"session": "cnt"})
    dfs.append(df.to_pandas())

pair_df = pd.concat(dfs)
pair_df = pair_df.groupby(["aid_x", "aid_y"])["cnt"].sum().reset_index()
pair_df.to_pickle(PREP_DIR + f"co_visitation_matrix_orders.pkl")


100%|██████████| 200/200 [00:08<00:00, 23.24it/s]


In [7]:
line_notify.send("prep_co_visitaion_matrix_orders end")